# Goal

I'm working with non spatially-referenced images produced from a laser ablation system. Along with the image, the software produces an "align" file of the same name - an xml file that contains the XY center point of each image (coordinates are in microns) and the XY size (in microns) along with any rotation. 

I would like to display the images (we typically generate about 1000 images / day) in their correct relative positions and sizes.

The laser has a stage with a cartesian positive XY coordinate system (in microns), and the origin in the SW corner. 

Is there a way to either turn the images into georeferenced geotiffs (using the information in the xml file)? So far I have gotten to the step of generating an affine transform for each scanned image based on it's size and center coordinate in microns and rotation metadata. What I don't have is a definition of a custom CRS in WKT. I don't have any experience with the WKt format but it seems like the only way to represent non-Earth CRSs.

Are there any resources for defining a custom CRS in WKT? I know the units are in microns, the center point of my datum, and the height and width of the coordinate reference system.


### Matching png paths to metadata paths and extracting the metadata into tabular format

In [ ]:
import pandas as pd
import skimage.io as skio
from pathlib import Path
import xml.etree.ElementTree as et 
import numpy as np
import affine
import os
from osgeo import gdal, osr

def match_png_align(png_file_path, align_file_path):
    """Tests if png and align file IDs are the same."""
    png_lst = str(png_file_path.name).split("_")
    align_lst = str(align_file_path.name).split("_")
    if png_lst[1] == align_lst[1] \
    and png_lst[2] == align_lst[2] \
    and png_lst[3] == align_lst[3] \
    and png_lst[-1][:-4] == align_lst[-1][:-6]:
        return (png_file_path, align_file_path)


def read_transform_inputs_png(meta_path):
    """Reads xml info about the scanned image used to transform coordinates"""
    xtree = et.parse(meta_path)
    root = xtree.getroot()
    center_info = root[0][2].text.split(",")
    size_info = root[0][3].text.split(",")
    extra_info = root[0][0].text.split(";")
    return {"rotation": float(root[0][1].text), 
           "center_x": float(center_info[0]),
           "center_y" : float(center_info[1]),
           "size_x" : float(size_info[0]),
           "size_y" : float(size_info[1]),
           "brightness": float(extra_info[0].split("=")[1]),
           "contrast": float(extra_info[1].split("=")[1]),
           "autoexposure": float(extra_info[2].split("=")[1]),
           "exposuretime": float(extra_info[3].split("=")[1])}

def read_transform_inputs_datum(datum_path):
    parser = et.XMLParser(encoding="utf-8")
    xtree = et.parse(datum_path, parser=parser)
    root = xtree.getroot()
    return {
        "rotation" : float(root[0][0].text),
        "center_x" : float(root[0][1].text.split(",")[0]),
        "center_y" : float(root[0][1].text.split(",")[1]),
        "size_x" : float(root[0][2].text.split(",")[0]),
        "size_y" : float(root[0][2].text.split(",")[1]),
        "focus" : float(root[0][3].text)
    }

In [ ]:
images_p = Path("images/")

align_paths = list(images_p.glob("ScanImage*.Align"))
png_paths = list(images_p.glob("ScanImage*.png"))

matches = []
for a in align_paths:
    for p in png_paths:
        if match_png_align(p, a):
            matches.append((p,a))

image_df = pd.DataFrame(matches, columns=["png","meta"])


image_df = image_df.join(pd.json_normalize(image_df.meta.apply(read_transform_inputs_png)))

image_df['source_shape'] = image_df.png.apply(lambda x: skio.imread(x).shape)

image_df = image_df.join(pd.DataFrame(image_df['source_shape'].tolist(), index=image_df.index, columns=["source_size_y", "source_size_x", "source_size_band"])  )  

datum_im_p = list(images_p.glob("Image_1*"))[0]
datum_align_p = list(images_p.glob("Image_1*"))[1]

# Defining the transform for each png

The format for the transfrm is (uperleftx, scalex, skewx, uperlefty, skewy, scaley) and defines how to map pixel coordinates to CRS coordinates. The only thing missing is a custom CRS that is defined from the datum image metadata.

X and Y pixel resolution of the datum image

In [ ]:
image_df['resolution_x'] = image_df['size_x'] / image_df['source_size_x']
image_df['resolution_y'] = image_df['size_y'] / image_df['source_size_y']

Since we know the center and size in projection coordinate of each png scan, we can get the upper left.

In [ ]:
image_df['upleftx'] = image_df.center_x - image_df.size_x / 2
image_df['uplefty'] = image_df.center_y - image_df.size_y / 2
# Specify raster location through geotransform array
# (uperleftx, scalex, skewx, uperlefty, skewy, scaley)
image_df["affine_transform"] = image_df.apply(lambda row: affine.Affine(row['upleftx'], row['resolution_x'], row['rotation'], row['uplefty'], row['rotation'], -row['resolution_y']), axis=1)

According to John, for the datum, we need to find the x and y size AFTER rotation by half a degree, the sizes in the metadata are pre rotation.

The datum info here might not be necessary, except to reference the datum image since it is rotated.

In [ ]:
# for some reason this starts throwing a parse error. pulled the big image size and center out manually, put in georef func
# datum_meta = read_transform_inputs_datum(str(datum_align_p))
# datum_meta

# Georeferences each png on a cartesian projection (epsg:6507) based on the upper left coordinate in microns, then saves as a geotiff

In [ ]:
def georef(row, outfolder="output"):
    # might need these to define total bounds of the CRS but maybe not
    center_x = 42085.00
    center_y = 47699.00
    size_x = 117207.00
    size_y = 104307.00
    src_filename = str(row["png"])
    
    if os.path.exists(outfolder) == False:
        os.mkdir(outfolder)
    
    dst_filename = os.path.join(outfolder, os.path.basename(str(row['png'])).split(".")[0] + ".tif")
    
    # Opens source dataset
    src_ds = gdal.Open(src_filename)

    driver = gdal.GetDriverByName('GTiff')
    ds = driver.Create(dst_filename, xsize=int(row['source_size_x']), ysize=int(row['source_size_y']), bands=3)

    # this assumes the projection is Geographic lat/lon WGS 84
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(6507)
    ds.SetProjection(srs.ExportToWkt())

    gt = list(row['affine_transform'])[0:6]
    ds.SetGeoTransform(gt)

    outband = ds.GetRasterBand(1)
    arr = skio.imread(src_filename)
    arr = np.moveaxis(arr, -1, 0)
    for i, band in enumerate(arr, 1):
        ds.GetRasterBand(i).WriteArray(band)
        
# georef(image_df.iloc[1])

In [ ]:
image_df.apply(georef, axis=1)